In [ ]:
import sys

sys.path.append("..")
from piece import piece

piece.start(should_send_to_score=True)

In [ ]:
from soundmining_tools.supercollider_receiver import ExtendedNoteHandler, PatchArguments
from soundmining_tools.supercollider_client import SupercolliderClient
from soundmining_tools.modular.control_instruments import *
from soundmining_tools.note import *
from soundmining_tools.generative import *
from soundmining_tools.spectrum import *
from enum import Enum
from soundmining_tools.sequencer import Sequencer, SequenceNote
from soundmining_tools.ui.ui_piece import UiPieceBuilder
from ipycanvas import Canvas, hold_canvas
from soundmining_tools.modular.instrument import NodeId
from soundmining_tools import supercollider_client
from soundmining_tools.modular import instrument

piece.reset()
piece.synth_player.start()

if piece.synth_player.should_send_to_score:
    score = piece.synth_player.supercollider_score
    score.add_message(supercollider_client.group_head(0, instrument.NodeId.SOURCE.value))
    score.add_message(supercollider_client.group_tail(NodeId.SOURCE.value, NodeId.EFFECT.value))
    score.add_message(supercollider_client.group_tail(NodeId.EFFECT.value, NodeId.ROOM_EFFECT.value))
    score.add_message(supercollider_client.load_dir(instrument.DEFAULT_SYNTH_DIR))

def static_control(value: float) -> StaticControl:
    return piece.control_instruments.static_control(value)


def sine_control(start_value: float, peak_value: float) -> SineControl:
    return piece.control_instruments.sine_control(start_value, peak_value)


def line_control(start_value: float, end_value: float) -> LineControl:
    return piece.control_instruments.line_control(start_value, end_value)


def three_block_control(
    levels: tuple[float, float, float, float],
    times: tuple[float, float, float],
    curves: tuple[float, float, float],
) -> ThreeBLockControl:
    return piece.control_instruments.three_block_control(levels, times, curves)


fundamental = note_to_hertz("a4")
first_partial = note_to_hertz("fiss5")
fact = make_fact(fundamental, first_partial)
undertone_spectrum = make_undertone_spectrum(fundamental, fact, 50)
overtone_spectrum = make_spectrum(fundamental, fact, 50)

# sub 13 - 40
sub_partials = undertone_spectrum[13:40]
# low 0 - 13
low_partials = undertone_spectrum[1:13]
# middle 0 - 7
middle_partials = overtone_spectrum[0:7]
# high 7 - 17
high_partials = overtone_spectrum[7:17]
# ultra 17 - 35
ultra_partials = overtone_spectrum[17:35]

all_partials = (
    sub_partials + low_partials + middle_partials + high_partials + ultra_partials
)
min_partial = min(all_partials)
max_partial = max(all_partials)


class SequenceType(Enum):
    SUB = 1
    LOW = 2
    MIDDLE = 3
    HIGH = 4
    ULTRA = 5


low_partials_matrix = {
    SequenceType.LOW: {
        SequenceType.LOW: 0.6,
        SequenceType.SUB: 0.2,
        SequenceType.MIDDLE: 0.2,
    },
    SequenceType.SUB: {
        SequenceType.LOW: 0.6,
        SequenceType.SUB: 0.1,
        SequenceType.MIDDLE: 0.3,
    },
    SequenceType.MIDDLE: {
        SequenceType.LOW: 0.6,
        SequenceType.SUB: 0.3,
        SequenceType.MIDDLE: 0.1,
    },
}

low_partials_chain = MarkovChain(low_partials_matrix, SequenceType.LOW)

middle_partials_matrix = {
    SequenceType.MIDDLE: {
        SequenceType.MIDDLE: 0.6,
        SequenceType.LOW: 0.2,
        SequenceType.HIGH: 0.2,
    },
    SequenceType.LOW: {
        SequenceType.MIDDLE: 0.6,
        SequenceType.LOW: 0.1,
        SequenceType.HIGH: 0.3,
    },
    SequenceType.HIGH: {
        SequenceType.MIDDLE: 0.6,
        SequenceType.LOW: 0.3,
        SequenceType.HIGH: 0.1,
    },
}

middle_partials_chain = MarkovChain(middle_partials_matrix, SequenceType.MIDDLE)

high_partials_matrix = {
    SequenceType.HIGH: {
        SequenceType.HIGH: 0.6,
        SequenceType.MIDDLE: 0.2,
        SequenceType.ULTRA: 0.2,
    },
    SequenceType.MIDDLE: {
        SequenceType.HIGH: 0.6,
        SequenceType.MIDDLE: 0.1,
        SequenceType.ULTRA: 0.3,
    },
    SequenceType.ULTRA: {
        SequenceType.HIGH: 0.6,
        SequenceType.MIDDLE: 0.3,
        SequenceType.ULTRA: 0.1,
    },
}

high_partials_chain = MarkovChain(high_partials_matrix, SequenceType.HIGH)


def choose_freq(sequence_type: SequenceType) -> float:
    match sequence_type:
        case SequenceType.SUB:
            partials = sub_partials
        case SequenceType.LOW:
            partials = low_partials
        case SequenceType.MIDDLE:
            partials = middle_partials
        case SequenceType.HIGH:
            partials = high_partials
        case SequenceType.ULTRA:
            partials = ultra_partials
    return random.choice(partials)


def choose_pan_point(sequence_type: SequenceType) -> float:
    match sequence_type:
        case SequenceType.SUB:
            points = [(-0.2, 0.2)]
        case SequenceType.LOW:
            points = [(-0.4, -0.2), (0.2, 0.4)]
        case SequenceType.MIDDLE:
            points = [(-0.6, -0.4), (0.4, 0.6)]
        case SequenceType.HIGH:
            points = [(-0.8, -0.6), (0.6, 0.8)]
        case SequenceType.ULTRA:
            points = [(-0.99, -0.8), (0.8, 0.99)]
    return pan_point(points)


def choose_pan_line(sequence_type: SequenceType) -> tuple[float, float]:
    match sequence_type:
        case SequenceType.SUB:
            points = [(-0.2, 0.2)]
            distance = 0.2
        case SequenceType.LOW:
            points = [(-0.4, -0.2), (0.2, 0.4)]
            distance = 0.4
        case SequenceType.MIDDLE:
            points = [(-0.6, -0.4), (0.4, 0.6)]
            distance = 0.6
        case SequenceType.HIGH:
            points = [(-0.8, -0.6), (0.6, 0.8)]
            distance = 0.8
        case SequenceType.ULTRA:
            points = [(-0.99, -0.8), (0.8, 0.99)]
            distance = 1.2
    return pan_line(distance, points)


def choose_sequence_type_chain(sequence_type: SequenceType) -> MarkovChain:
    match sequence_type:
        case SequenceType.LOW:
            return low_partials_chain
        case SequenceType.MIDDLE:
            return middle_partials_chain
        case SequenceType.HIGH:
            return high_partials_chain

LOW_FM1_TRACK = "Low Fm1"
MIDDLE_FM1_TRACK = "Middle Fm1"
HIGH_FM1_TRACK = "High Fm1"
LOW_FM2_TRACK = "Low Fm2"
MIDDLE_FM2_TRACK = "Middle Fm2"
HIGH_FM2_TRACK = "High Fm2"
LOW_NOISE_TRACK = "Low Noise"
MIDDLE_NOISE_TRACK = "Middle Noise"
HIGH_NOISE_TRACK = "High Noise"

# https://www.youtube.com/watch?v=oR4VZy2LJ60
# BPF.ar(sig, freq, rq, 1/rq.sqrt)
# The reciprocal of Q. Q is conventionally defined as freq / bandwidth, meaning rq = (bandwidth / freq).

EFFECT_LENGTH = 60 * 15
NUMBER_OF_SEQUENCES = 34

def make_fm2_effect(effect_output: int, clean_output: int, effect_level: float = 0.5, clean_level: float = 0.5):
    short_effect_sound = piece.synth_player.note(NodeId.EFFECT).stereo_input()
    short_reverb_effect_sound = (
        piece.synth_player.note(NodeId.ROOM_EFFECT)
        .input_from_note(short_effect_sound)
        .stereo_volume(piece.control_instruments.static_control(effect_level))
        # .stereo_free_reverb(piece.control_instruments.three_block_control((0, 1, 1, 0), (0.01, 0.90, 0.09), (0, 0, 0)), mix=1.0, room=0.5, damp=0.4)
        .stereo_g_verb(
            piece.control_instruments.three_block_control(
                (0, 1, 1, 0), (0.01, 0.90, 0.09), (0, 0, 0)
            ),
            roomsize=75,
            revtime=4,
            damping=0.5,
            inputbw=0.5,
            earlyreflevel=0.5,
            taillevel=0.6,
        )
        # .stereo_high_pass_filter(piece.control_instruments.static_control(100))
        # .stereo_low_pass_filter(piece.control_instruments.static_control(2000))
    )
    short_reverb_effect_sound.play(0, EFFECT_LENGTH, output_bus=effect_output)

    short_clean_effect_sound = (
        piece.synth_player.note(NodeId.ROOM_EFFECT)
        .input_from_note(short_effect_sound)
        .stereo_volume(piece.control_instruments.static_control(clean_level))
    )
    short_clean_effect_sound.play(0, EFFECT_LENGTH, output_bus=clean_output)

    return short_effect_sound

def make_fm1_effect(effect_output: int, clean_output: int, effect_level: float = 0.5, clean_level: float = 0.5):
    short_effect_sound = piece.synth_player.note(NodeId.EFFECT).stereo_input()
    short_reverb_effect_sound = (
        piece.synth_player.note(NodeId.ROOM_EFFECT)
        .input_from_note(short_effect_sound)
        .stereo_volume(piece.control_instruments.static_control(effect_level))
        # .stereo_free_reverb(piece.control_instruments.three_block_control((0, 1, 1, 0), (0.01, 0.90, 0.09), (0, 0, 0)), mix=1.0, room=0.5, damp=0.4)
        .stereo_g_verb(
            piece.control_instruments.three_block_control(
                (0, 1, 1, 0), (0.01, 0.90, 0.09), (0, 0, 0)
            ),
            roomsize=50,
            revtime=0.5,
            damping=0.7,
            inputbw=0.5,
            earlyreflevel=0.4,
            taillevel=0.5,
        )
        # .stereo_high_pass_filter(piece.control_instruments.static_control(100))
        # .stereo_low_pass_filter(piece.control_instruments.static_control(2000))
    )
    short_reverb_effect_sound.play(0, EFFECT_LENGTH, output_bus=effect_output)

    short_clean_effect_sound = (
        piece.synth_player.note(NodeId.ROOM_EFFECT)
        .input_from_note(short_effect_sound)
        .stereo_volume(piece.control_instruments.static_control(clean_level))
    )
    short_clean_effect_sound.play(0, EFFECT_LENGTH, output_bus=clean_output)

    return short_effect_sound

def make_noise_effect(effect_output: int, clean_output: int, effect_level: float = 0.5, clean_level: float = 0.5):
    long_effect_sound = piece.synth_player.note(NodeId.EFFECT).stereo_input()

    long_reverb_effect_sound = (
        piece.synth_player.note(NodeId.ROOM_EFFECT)
        .input_from_note(long_effect_sound)
        .stereo_volume(piece.control_instruments.static_control(effect_level))
        .stereo_g_verb(
            piece.control_instruments.three_block_control(
                (0, 1, 1, 0), (0.01, 0.90, 0.09), (0, 0, 0)
            ),
            roomsize=150,
            revtime=13,
            damping=0.5,
            inputbw=0.5,
            earlyreflevel=0.5,
            taillevel=0.7,
        )
        .stereo_high_pass_filter(piece.control_instruments.static_control(100))
        .stereo_low_pass_filter(piece.control_instruments.static_control(1000))
    )

    long_reverb_effect_sound.play(0, EFFECT_LENGTH, output_bus=effect_output)

    long_clean_effect_sound = (
        piece.synth_player.note(NodeId.ROOM_EFFECT)
        .input_from_note(long_effect_sound)
        .stereo_volume(piece.control_instruments.static_control(clean_level))
    )
    long_clean_effect_sound.play(0, EFFECT_LENGTH, output_bus=clean_output)
    return long_effect_sound


LOW_FM1_OUTPUT_CLEAN = 0
LOW_FM1_OUTPUT_EFFECT = 2
MIDDLE_FM1_OUTPUT_CLEAN = 4
MIDDLE_FM1_OUTPUT_EFFECT = 6
HIGH_FM1_OUTPUT_CLEAN = 8
HIGH_FM1_OUTPUT_EFFECT = 10

LOW_FM2_OUTPUT_CLEAN = 12
LOW_FM2_OUTPUT_EFFECT = 14
MIDDLE_FM2_OUTPUT_CLEAN = 16
MIDDLE_FM2_OUTPUT_EFFECT = 18
HIGH_FM2_OUTPUT_CLEAN = 20
HIGH_FM2_OUTPUT_EFFECT = 22

LOW_NOISE_OUTPUT_CLEAN = 24
LOW_NOISE_OUTPUT_EFFECT = 26
MIDDLE_NOISE_OUTPUT_CLEAN = 28
MIDDLE_NOISE_OUTPUT_EFFECT = 30
HIGH_NOISE_OUTPUT_CLEAN = 32
HIGH_NOISE_OUTPUT_EFFECT = 34

low_fm1_effect = make_fm1_effect(LOW_FM1_OUTPUT_EFFECT, LOW_FM1_OUTPUT_CLEAN, effect_level=0.6, clean_level=0.9)
middle_fm1_effect = make_fm1_effect(MIDDLE_FM1_OUTPUT_EFFECT, MIDDLE_FM1_OUTPUT_CLEAN, effect_level=0.7, clean_level=1.1)
high_fm1_effect = make_fm1_effect(HIGH_FM1_OUTPUT_EFFECT, HIGH_FM1_OUTPUT_CLEAN, effect_level=0.6, clean_level=1)

low_fm2_effect = make_fm2_effect(LOW_FM2_OUTPUT_EFFECT, LOW_FM2_OUTPUT_CLEAN, effect_level=2.3, clean_level=5)
middle_fm2_effect = make_fm2_effect(MIDDLE_FM2_OUTPUT_EFFECT, MIDDLE_FM2_OUTPUT_CLEAN, effect_level=2, clean_level=5.3)
high_fm2_effect = make_fm2_effect(HIGH_FM2_OUTPUT_EFFECT, HIGH_FM2_OUTPUT_CLEAN, effect_level=2.3, clean_level=4.5)

low_noise_effect = make_noise_effect(LOW_NOISE_OUTPUT_EFFECT, LOW_NOISE_OUTPUT_CLEAN, effect_level=2, clean_level=2)
middle_noise_effect = make_noise_effect(MIDDLE_NOISE_OUTPUT_EFFECT, MIDDLE_NOISE_OUTPUT_CLEAN, effect_level=3, clean_level=3)
high_noise_effect = make_noise_effect(HIGH_NOISE_OUTPUT_EFFECT, HIGH_NOISE_OUTPUT_CLEAN, effect_level=2, clean_level=2)

def fm_note_1(
    start_time: float, sequence_type: SequenceType, duration: float
) -> SequenceNote:
    sequence_type_chain = choose_sequence_type_chain(sequence_type)
    match sequence_type:
        case SequenceType.LOW:
            track = LOW_FM1_TRACK
            static_amp_factor = 1.5
            effect = low_fm1_effect
        case SequenceType.MIDDLE:
            track = MIDDLE_FM1_TRACK
            static_amp_factor = 1.2
            effect = middle_fm1_effect
        case SequenceType.HIGH:
            track = HIGH_FM1_TRACK
            static_amp_factor = 1.5
            effect = high_fm1_effect
    note_freq = choose_freq(sequence_type_chain.next())
    mod_amount = random_range(300, 10000)
    random_sideband_freq = choose_freq(sequence_type_chain.next())
    random_ring_mod_freq = choose_freq(sequence_type_chain.next())
    pan_pos = choose_pan_point(sequence_type_chain.next())
    amp_control = three_block_control(
        (
            0,
            random_range(0.02, 0.03) * static_amp_factor,
            random_range(0.02, 0.03) * static_amp_factor,
            0,
        ),
        (0.2, 0.5, 0.3),
        (4, 0, -4),
    )
    # print(f"fm note 1 start {start_time} duration {duration} carrier {note_freq} sideband {random_sideband_freq} mod amount = {mod_amount} ring {random_ring_mod_freq} pan {pan_pos}")
    (
        piece.synth_player.note()
        # REALLY GOOD
        .saw(static_control(random_sideband_freq), static_control(mod_amount))
        .fm_pulse_modulate(static_control(note_freq), static_control(1))
        .ring_modulate(static_control(random_ring_mod_freq))
        .mono_volume(amp_control)
        .pan(static_control(pan_pos))
        # .play(start_time, duration)
        .send_to_synth_note(effect, start_time, duration)
    )
    return SequenceNote(
        start=start_time, track=track, duration=duration, freq=note_freq
    )



def fm_note_2(
    start_time: float, sequence_type: SequenceType, duration: float
) -> SequenceNote:
    sequence_type_chain = choose_sequence_type_chain(sequence_type)
    match sequence_type:
        case SequenceType.LOW:
            static_amp_factor = 2.7
            track = LOW_FM2_TRACK
            effect = low_fm2_effect
        case SequenceType.MIDDLE:
            static_amp_factor = 2.6
            track = MIDDLE_FM2_TRACK
            effect = middle_fm2_effect
        case SequenceType.HIGH:
            static_amp_factor = 2.5
            track = HIGH_FM2_TRACK
            effect = high_fm2_effect
    note_freq = choose_freq(sequence_type_chain.next())
    mod_amount = random_range(300, 10000)
    random_sideband_freq = choose_freq(sequence_type_chain.next())
    random_ring_mod_freq = choose_freq(sequence_type_chain.next())
    pan_pos = choose_pan_point(sequence_type_chain.next())
    amp_control = three_block_control(
        (
            0,
            random_range(0.02, 0.03) * static_amp_factor,
            random_range(0.02, 0.03) * static_amp_factor,
            0,
        ),
        (0.2, 0.5, 0.3),
        (4, 0, -4),
    )
    # print(f"fm note 2 start {start_time} duration {duration} carrier {note_freq} mod amount {mod_amount} sideband {random_sideband_freq} ring {random_ring_mod_freq}")
    (
        piece.synth_player.note()
        # REALLY GOOD
        .triangle(static_control(random_sideband_freq), static_control(mod_amount))
        .fm_saw_modulate(static_control(note_freq), static_control(1))
        .ring_modulate(static_control(random_ring_mod_freq))
        .mono_volume(amp_control)
        .pan(static_control(pan_pos))
        .send_to_synth_note(effect, start_time, duration)
        # .play(start_time, duration)
    )
    return SequenceNote(
        start=start_time, track=track, duration=duration, freq=note_freq
    )


def filtered_noise(
    start_time: float, sequence_type: SequenceType, duration: float
) -> SequenceNote:
    sequence_type_chain = choose_sequence_type_chain(sequence_type)
    match sequence_type:
        case SequenceType.LOW:
            static_amp_factor = 2
            track = LOW_NOISE_TRACK
            effect = low_noise_effect
        case SequenceType.MIDDLE:
            static_amp_factor = 1
            track = MIDDLE_NOISE_TRACK
            effect = middle_noise_effect
        case SequenceType.HIGH:
            static_amp_factor = 1
            track = HIGH_NOISE_TRACK
            effect = high_noise_effect
    note_freq = choose_freq(sequence_type_chain.next())
    random_sideband = choose_freq(sequence_type_chain.next())
    pan_line = choose_pan_line(sequence_type_chain.next())

    bw = random_range(50, 75)
    # bw = random_range(100, 200)
    # bw = random_range(300, 400)
    rq = bw / note_freq
    amp_factor = (1 / math.sqrt(rq)) * static_amp_factor

    # print(f"Filtered noise start {start_time} duration {duration} filter freq {note_freq} ring {random_sideband} and bw {bw} rq {rq} amp factor {amp_factor} pan {pan_line}")
    (
        piece.synth_player.note()
        .white_noise(sine_control(0, random_range(0.5, 0.99)))
        .mono_band_pass_filter(static_control(note_freq), static_control(rq))
        .mono_volume(static_control(amp_factor))
        .ring_modulate(static_control(random_sideband))
        .pan(line_control(pan_line[0], pan_line[1]))
        .send_to_synth_note(effect, start_time, duration)
        # .play(start_time, duration)
    )
    return SequenceNote(
        start=start_time, track=track, duration=duration, freq=note_freq
    )


class NoteLength(Enum):
    LONG = 1
    MIDDLE = 2
    SHORT = 3


filtered_noise_note_length_matrix = {
    NoteLength.SHORT: {NoteLength.SHORT: 0.7, NoteLength.MIDDLE: 0.3},
    NoteLength.MIDDLE: {NoteLength.SHORT: 0.95, NoteLength.MIDDLE: 0.05},
}
filtered_noise_note_length_chain = MarkovChain(
    filtered_noise_note_length_matrix, NoteLength.SHORT
)


def filtered_noise_burst(
    start_time: float, seqence_type: SequenceType, duration: float
) -> SequenceNote:
    notes = []
    current_time = start_time
    for _ in range(random_int_range(2, 5)):
        notes.append(filtered_noise(current_time, seqence_type, duration))
        match filtered_noise_note_length_chain.next():
            case NoteLength.SHORT:
                note_len = random_range(0.1, 0.2)
            case NoteLength.MIDDLE:
                note_len = random_range(0.5, 1)
        current_time += note_len
    return notes


class ExploreFilteredNoiseBw(ExtendedNoteHandler):
    def __init__(self, client: SupercolliderClient) -> None:
        super().__init__(client)

    def handle_note(self, patch_arguments: PatchArguments) -> None:
        match patch_arguments.octave:
            case 2:
                match patch_arguments.note:
                    case 0:
                        sequence_type = SequenceType.LOW
                    case 1:
                        sequence_type = SequenceType.MIDDLE
                    case 2:
                        sequence_type = SequenceType.HIGH
                fm_note_1(patch_arguments.start, sequence_type, random_range(13, 21))
            case 3:
                match patch_arguments.note:
                    case 0:
                        sequence_type = SequenceType.LOW
                    case 1:
                        sequence_type = SequenceType.MIDDLE
                    case 2:
                        sequence_type = SequenceType.HIGH
                fm_note_2(patch_arguments.start, sequence_type, random_range(13, 21))
            case 4:
                match patch_arguments.note:
                    case 0:
                        sequence_type = SequenceType.LOW
                    case 1:
                        sequence_type = SequenceType.MIDDLE
                    case 2:
                        sequence_type = SequenceType.HIGH
                # filtered_noise(patch_arguments.start, sequence_type, random_range(3, 5))
                filtered_noise_burst(
                    patch_arguments.start, sequence_type, random_range(3, 5)
                )


my_handler = ExploreFilteredNoiseBw(piece.supercollider_client)
piece.receiver.set_note_handler(my_handler)

play_noise_burst_matrix = {True: {True: 0, False: 1}, False: {True: 0.2, False: 0.80}}
play_noise_burst_chain = MarkovChain(play_noise_burst_matrix, False)


def fm1_step_handler(
    i: int, time: float, sequence_type: SequenceType, duration: float
) -> list[SequenceNote]:
    notes = []
    current_time = time
    stop_time = current_time + duration
    while current_time < stop_time:
        for _ in range(random_int_range(2, 5)):
            notes.append(
                fm_note_1(
                    current_time + random_range(0, 1),
                    sequence_type,
                    random_range(21, 34),
                )
            )
        if play_noise_burst_chain.next():
            notes.extend(
                filtered_noise_burst(
                    current_time + (random_range(21, 34) * random_range(0.3, 0.6)),
                    sequence_type,
                    random_range(3, 5),
                )
            )
        current_time += random_range(8, 13)
    return notes


def fm2_step_handler(
    i: int, time: float, sequence_type: SequenceType, duration: float
) -> list[SequenceNote]:
    notes = []
    current_time = time
    stop_time = current_time + duration
    while current_time < stop_time:
        for _ in range(random_int_range(2, 5)):
            notes.append(
                fm_note_2(
                    current_time + random_range(0, 1),
                    sequence_type,
                    random_range(21, 34),
                )
            )
        if play_noise_burst_chain.next():
            notes.extend(
                filtered_noise_burst(
                    current_time + (random_range(21, 34) * random_range(0.3, 0.6)),
                    sequence_type,
                    random_range(3, 5),
                )
            )
        current_time += random_range(8, 13)
    return notes


class FmType(Enum):
    FM1 = 1
    FM2 = 2


part_type_matrix = {
    (FmType.FM1, SequenceType.MIDDLE): {
        (FmType.FM1, SequenceType.HIGH): 0.2,
        (FmType.FM1, SequenceType.LOW): 0.2,
        (FmType.FM2, SequenceType.HIGH): 0.2,
        (FmType.FM2, SequenceType.MIDDLE): 0.2,
        (FmType.FM2, SequenceType.LOW): 0.2,
    },
    (FmType.FM1, SequenceType.HIGH): {
        (FmType.FM1, SequenceType.MIDDLE): 0.2,
        (FmType.FM1, SequenceType.LOW): 0.2,
        (FmType.FM2, SequenceType.HIGH): 0.2,
        (FmType.FM2, SequenceType.MIDDLE): 0.2,
        (FmType.FM2, SequenceType.LOW): 0.2,
    },
    (FmType.FM1, SequenceType.LOW): {
        (FmType.FM1, SequenceType.HIGH): 0.2,
        (FmType.FM1, SequenceType.MIDDLE): 0.2,
        (FmType.FM2, SequenceType.HIGH): 0.2,
        (FmType.FM2, SequenceType.MIDDLE): 0.2,
        (FmType.FM2, SequenceType.LOW): 0.2,
    },
    (FmType.FM2, SequenceType.MIDDLE): {
        (FmType.FM2, SequenceType.HIGH): 0.2,
        (FmType.FM2, SequenceType.LOW): 0.2,
        (FmType.FM1, SequenceType.HIGH): 0.2,
        (FmType.FM1, SequenceType.MIDDLE): 0.2,
        (FmType.FM1, SequenceType.LOW): 0.2,
    },
    (FmType.FM2, SequenceType.HIGH): {
        (FmType.FM2, SequenceType.MIDDLE): 0.2,
        (FmType.FM2, SequenceType.LOW): 0.2,
        (FmType.FM1, SequenceType.HIGH): 0.2,
        (FmType.FM1, SequenceType.MIDDLE): 0.2,
        (FmType.FM1, SequenceType.LOW): 0.2,
    },
    (FmType.FM2, SequenceType.LOW): {
        (FmType.FM2, SequenceType.HIGH): 0.2,
        (FmType.FM2, SequenceType.MIDDLE): 0.2,
        (FmType.FM1, SequenceType.HIGH): 0.2,
        (FmType.FM1, SequenceType.MIDDLE): 0.2,
        (FmType.FM1, SequenceType.LOW): 0.2,
    },
}

part_type_chain = MarkovChain(part_type_matrix, (FmType.FM1, SequenceType.MIDDLE))

from typing import Callable


def make_step_handler(
    fm_type: FmType, sequence_type: SequenceType, duration: float
) -> Callable[[int, float], list[SequenceNote]]:
    match fm_type:
        case FmType.FM1:
            step_handler = fm1_step_handler
        case FmType.FM2:
            step_handler = fm2_step_handler
    return lambda i, time: step_handler(i, time, sequence_type, duration)


last_sequenser: Sequencer = None
for _ in range(NUMBER_OF_SEQUENCES):
    fm_type, sequence_type = part_type_chain.next()

    next_time = 60 * random_range(0.85, 0.66)
    start_time = next_time * random_range(0.5, 0.75)

    step_handler = make_step_handler(fm_type, sequence_type, 55)
    new_sequencer = (
        Sequencer(1)
        .add_step_handler(step_handler)
        .next_time_handler(lambda i: next_time)
    )

    if last_sequenser:
        new_sequencer.spawn_sequencer(0, last_sequenser)
        last_sequenser.start_time_handler(lambda start: start + start_time)
    last_sequenser = new_sequencer


notes = last_sequenser.generate(0)
# notes = []

if piece.synth_player.should_send_to_score:
    piece.synth_player.supercollider_score.make_score_file("module-music-8-v1.txt")

ui_piece = UiPieceBuilder().add_notes(notes).build()

piece_duration = ui_piece.get_duration()

piece_stats = {"total": piece_duration, "total minutes": piece_duration / 60.0}

for track in ui_piece.tracks:
    track_duration = 0
    for note in track.notes:
        track_duration = max(track_duration, note.start + note.duration)
    piece_stats[track.track_name] = track_duration

display(piece_stats)

# https://github.com/jupyter-widgets/jupyterlab-sidecar
# try sidecar to display in a separate window
# MultiCanvas to have background and foreground

# TRACK_HEIGHT = 200
# NOTE_SCALE_FACTOR = 10
# HEIGHT_INDENT = 180

TRACK_HEIGHT = 100
NOTE_SCALE_FACTOR = 2
HEIGHT_INDENT = 80

ui_width = 200 + (piece_duration * NOTE_SCALE_FACTOR)
ui_height = TRACK_HEIGHT * len(ui_piece.tracks)
min_freq = min_partial
max_freq = max_partial

stop_animation = False
import time
from ipywidgets import Output

canvas = Canvas(width=ui_width, height=ui_height)

out = Output()


@out.capture()
def handle_mouse_down(x, y):
    canvas.flush()
    print("Mouse down event:", x, y)


canvas.on_mouse_down(handle_mouse_down)
canvas.global_alpha = 0.7

display(canvas)

with hold_canvas():
    start_drawing_time = int(time.time())

    canvas.clear()
    for track_index, track in enumerate(ui_piece.tracks):
        canvas.font = "18px sans-serif"
        canvas.fill_style = "Black"
        canvas.fill_text(
            track.track_name, x=20, y=(track_index * TRACK_HEIGHT) + HEIGHT_INDENT
        )
        canvas.stroke_style = "Black"
        canvas.stroke_lines(
            [
                (150, (track_index * TRACK_HEIGHT) + 10),
                (150, ((track_index * TRACK_HEIGHT) + TRACK_HEIGHT - 10)),
            ]
        )
        for note in track.notes:
            relative_note = (note.freq - min_freq) / (max_freq - min_freq)
            startx = 200 + (note.start * NOTE_SCALE_FACTOR)
            starty = (
                (track_index * TRACK_HEIGHT)
                - (relative_note * HEIGHT_INDENT)
                + HEIGHT_INDENT
            )
            peakx = 200 + (note.start + (note.duration * note.peak)) * NOTE_SCALE_FACTOR
            peaky = (
                (track_index * TRACK_HEIGHT)
                - (relative_note * HEIGHT_INDENT)
                + HEIGHT_INDENT
                - 5
            )
            endx = 200 + (note.start + note.duration) * NOTE_SCALE_FACTOR
            endy = (
                (track_index * TRACK_HEIGHT)
                - (relative_note * HEIGHT_INDENT)
                + HEIGHT_INDENT
            )
            canvas.stroke_style = note.color
            canvas.stroke_lines([(startx, starty), (peakx, peaky), (endx, endy)])

import ipywidgets as widgets

stop_button = widgets.Button(description="Stop")
status = widgets.Output()
display(stop_button, status)
with status:
    print("Playing")


def stop_playback(b):
    piece.reset()
    canvas.clear()
    status.clear_output()
    with status:
        print("Playback stopped")


stop_button.on_click(stop_playback)

In [ ]:
piece.stop()